# Day 6-2: SQL 연동 간단히 (pandas & SQLAlchemy) - 정답

## 📚 학습 목표

**Part 1: 기초 (SQLite + pandas)**
1. SQLite 데이터베이스 생성 및 연결
2. `pd.read_sql()`로 SQL 쿼리 결과 DataFrame으로 읽기
3. `to_sql()`로 DataFrame을 SQL 테이블로 저장
4. 기본 SQL 쿼리 (SELECT, WHERE, GROUP BY) 실행
5. pandas와 SQL의 역할 분담 이해

**Part 2: 심화 (SQLAlchemy + BigQuery)**
1. SQLAlchemy로 다양한 DB 연결 (PostgreSQL, MySQL)
2. Connection 관리 및 트랜잭션 처리
3. pandas-gbq로 BigQuery 데이터 읽기/쓰기
4. 실무 워크플로우: 데이터 수집 → DB 저장 → 분석

---

## 🎯 실습 퀴즈 정답

---

### Q1. SQLite DB 생성 및 DataFrame 저장 ⭐

**문제**: 다음 고객 데이터를 `customers.db`라는 SQLite 데이터베이스의 `customers` 테이블로 저장하세요.

In [ ]:
# Q1 정답

import pandas as pd
import sqlite3

customers = pd.DataFrame({
    '고객ID': [101, 102, 103, 104, 105],
    '이름': ['김철수', '이영희', '박민수', '최지영', '정우진'],
    '가입일': ['2024-01-15', '2024-02-20', '2024-03-10', '2024-04-05', '2024-05-12'],
    '등급': ['Gold', 'Silver', 'Gold', 'Bronze', 'Silver']
})

# 1. SQLite DB 연결 (파일이 없으면 자동 생성)
conn = sqlite3.connect('customers.db')

# 2. DataFrame을 SQL 테이블로 저장
customers.to_sql('customers', conn, if_exists='replace', index=False)

# 3. 연결 종료
conn.close()

print("✅ customers.db 파일 생성 및 데이터 저장 완료")

In [ ]:
# 테스트: 저장된 데이터 확인
conn = sqlite3.connect('customers.db')
result = pd.read_sql("SELECT * FROM customers", conn)
conn.close()

assert len(result) == 5, "5행의 데이터가 저장되어야 합니다"
assert list(result.columns) == ['고객ID', '이름', '가입일', '등급'], "컬럼명이 일치해야 합니다"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. `sqlite3.connect()`로 DB 연결 (파일이 없으면 자동 생성)
2. `to_sql()`로 DataFrame을 테이블로 저장
3. `if_exists='replace'`로 기존 테이블이 있으면 덮어쓰기

**핵심 개념**:
- SQLite는 파일 기반 DB로, 별도 서버 설치 불필요
- `to_sql()`의 `index=False` 옵션은 pandas의 인덱스를 저장하지 않음

**대안 솔루션**:
```python
# SQLAlchemy 사용
from sqlalchemy import create_engine
engine = create_engine('sqlite:///customers.db')
customers.to_sql('customers', engine, if_exists='replace', index=False)
```

**흔한 실수**:
- ❌ `conn.close()` 생략: 파일 잠금 문제 발생 가능
- ❌ `if_exists` 파라미터 누락: 테이블이 이미 있으면 에러

**실무 팁**:
웹 스크레이핑한 데이터를 SQLite에 저장하면, Git에 올려 팀원들과 쉽게 공유할 수 있습니다.

---

### Q2. SELECT 쿼리로 데이터 읽기 ⭐

**문제**: Q1에서 생성한 `customers` 테이블에서 모든 데이터를 읽어 DataFrame으로 출력하세요.

In [ ]:
# Q2 정답

# 1. DB 연결
conn = sqlite3.connect('customers.db')

# 2. SELECT 쿼리 실행
query = "SELECT * FROM customers"
df = pd.read_sql(query, conn)

# 3. 연결 종료
conn.close()

print("고객 데이터:")
print(df)

In [ ]:
# 테스트
assert isinstance(df, pd.DataFrame), "결과는 DataFrame이어야 합니다"
assert df.shape == (5, 4), "5행 4열의 DataFrame이어야 합니다"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. `pd.read_sql()`에 SQL 쿼리와 연결 객체 전달
2. `SELECT *`는 모든 컬럼 조회
3. 결과는 자동으로 pandas DataFrame으로 변환됨

**핵심 개념**:
- `pd.read_sql()`은 SQL 쿼리 결과를 DataFrame으로 변환하는 핵심 함수
- SQL의 컬럼 순서가 DataFrame의 컬럼 순서로 유지됨

**대안 솔루션**:
```python
# pd.read_sql_table() 사용 (테이블명만 지정)
df = pd.read_sql_table('customers', conn)
```

**흔한 실수**:
- ❌ SQL 문법 오류: `SELECT * customers` (FROM 누락)
- ❌ 연결 객체 전달 안 함: `pd.read_sql(query)` → 에러

**실무 팁**:
`SELECT *`는 개발 단계에서만 사용하고, 프로덕션에서는 필요한 컬럼만 지정하는 것이 효율적입니다.

---

### Q3. WHERE 조건 필터링 ⭐⭐

**문제**: `customers` 테이블에서 등급이 'Gold'인 고객만 조회하는 SQL 쿼리를 작성하고, 결과를 DataFrame으로 출력하세요.

In [ ]:
# Q3 정답

conn = sqlite3.connect('customers.db')

# WHERE 절로 필터링
query = """
SELECT *
FROM customers
WHERE 등급 = 'Gold'
"""

df_gold = pd.read_sql(query, conn)
conn.close()

print("Gold 등급 고객:")
print(df_gold)

In [ ]:
# 테스트
assert len(df_gold) == 2, "Gold 등급 고객은 2명이어야 합니다"
assert all(df_gold['등급'] == 'Gold'), "모든 행의 등급이 Gold여야 합니다"
assert set(df_gold['이름']) == {'김철수', '박민수'}, "김철수, 박민수가 조회되어야 합니다"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. `WHERE 등급 = 'Gold'` 조건으로 필터링
2. SQL에서 문자열은 작은따옴표(`'`)로 감싸야 함

**핵심 개념**:
- WHERE 절은 행 단위 필터링 (pandas의 boolean indexing과 유사)
- SQL에서 먼저 필터링하면 네트워크 전송량 감소 (대용량 DB에서 중요)

**대안 솔루션**:
```python
# pandas로 필터링 (SQL 대신)
df = pd.read_sql("SELECT * FROM customers", conn)
df_gold = df[df['등급'] == 'Gold']
```

**흔한 실수**:
- ❌ 큰따옴표 사용: `WHERE 등급 = "Gold"` (SQLite는 허용하지만 비표준)
- ❌ 비교 연산자 오류: `WHERE 등급 == 'Gold'` (SQL은 `=` 사용)

**실무 팁**:
대용량 DB에서는 SQL WHERE로 먼저 필터링 후 pandas로 가져오는 것이 메모리 효율적입니다.

---

### Q4. 주문 데이터 추가 저장 ⭐⭐

**문제**: 주문 데이터를 `customers.db`의 `orders` 테이블로 저장하세요.

In [ ]:
# Q4 정답

orders = pd.DataFrame({
    '주문ID': [1001, 1002, 1003, 1004, 1005, 1006],
    '고객ID': [101, 102, 101, 103, 104, 102],
    '주문금액': [150000, 80000, 200000, 120000, 50000, 95000],
    '주문일': ['2024-06-01', '2024-06-02', '2024-06-03', '2024-06-04', '2024-06-05', '2024-06-06']
})

# 1. DB 연결
conn = sqlite3.connect('customers.db')

# 2. orders 테이블 저장
orders.to_sql('orders', conn, if_exists='replace', index=False)

# 3. 연결 종료
conn.close()

print("✅ orders 테이블 생성 완료")

In [ ]:
# 테스트
conn = sqlite3.connect('customers.db')
result = pd.read_sql("SELECT * FROM orders", conn)
conn.close()

assert len(result) == 6, "6행의 주문 데이터가 있어야 합니다"
assert '주문ID' in result.columns, "주문ID 컬럼이 있어야 합니다"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. 이미 존재하는 DB 파일(`customers.db`)에 연결
2. 새로운 테이블(`orders`) 추가 저장
3. 같은 DB에 여러 테이블 저장 가능

**핵심 개념**:
- 하나의 DB 파일에 여러 테이블을 저장할 수 있음 (관계형 DB의 기본 구조)
- `고객ID`가 외래키(Foreign Key) 역할을 하여 `customers` 테이블과 연결됨

**대안 솔루션**:
```python
# append 모드로 기존 데이터에 추가
new_orders = pd.DataFrame(...)
new_orders.to_sql('orders', conn, if_exists='append', index=False)
```

**흔한 실수**:
- ❌ 매번 새 DB 파일 생성: `connect('orders.db')` (잘못된 파일명)
- ❌ `if_exists` 미지정: 테이블이 이미 있으면 에러 발생

**실무 팁**:
Week 2-3 프로젝트에서 웹 스크레이핑 결과를 여러 테이블에 나누어 저장하면, 데이터 관리가 체계적입니다.

---

### Q5. GROUP BY 집계 쿼리 ⭐⭐⭐

**문제**: `orders` 테이블에서 고객별 총 주문금액과 주문 횟수를 계산하는 SQL 쿼리를 작성하세요.

In [ ]:
# Q5 정답

conn = sqlite3.connect('customers.db')

# GROUP BY로 고객별 집계
query = """
SELECT 
    고객ID,
    SUM(주문금액) as 총주문금액,
    COUNT(*) as 주문횟수
FROM orders
GROUP BY 고객ID
ORDER BY 총주문금액 DESC
"""

df_summary = pd.read_sql(query, conn)
conn.close()

print("고객별 주문 요약:")
print(df_summary)

In [ ]:
# 테스트
assert len(df_summary) == 4, "4명의 고객이 주문했습니다"
assert df_summary.iloc[0]['고객ID'] == 101, "고객ID 101이 가장 많이 주문했습니다"
assert df_summary.iloc[0]['총주문금액'] == 350000, "고객ID 101의 총 주문금액은 350,000원입니다"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. `GROUP BY 고객ID`로 고객별로 그룹화
2. `SUM(주문금액)`로 총 주문금액 계산
3. `COUNT(*)`로 주문 횟수 계산
4. `ORDER BY 총주문금액 DESC`로 내림차순 정렬

**핵심 개념**:
- GROUP BY는 pandas의 `groupby()`와 동일한 개념
- 집계 함수: SUM, COUNT, AVG, MAX, MIN
- SQL은 대용량 데이터 집계에 최적화되어 있음

**대안 솔루션**:
```python
# pandas groupby로 동일한 결과
df = pd.read_sql("SELECT * FROM orders", conn)
df_summary = df.groupby('고객ID').agg(
    총주문금액=('주문금액', 'sum'),
    주문횟수=('주문ID', 'count')
).reset_index().sort_values('총주문금액', ascending=False)
```

**흔한 실수**:
- ❌ GROUP BY 없이 집계 함수 사용: `SELECT SUM(주문금액)` → 전체 합계만 계산됨
- ❌ SELECT 절에 GROUP BY 컬럼 외 컬럼 포함: `SELECT 고객ID, 주문일, SUM(...)` → 에러

**실무 팁**:
수백만 건의 데이터를 집계할 때는 SQL GROUP BY가 pandas groupby보다 훨씬 빠릅니다.

---

### Q6. JOIN 쿼리로 테이블 결합 ⭐⭐⭐

**문제**: `customers`와 `orders` 테이블을 `고객ID`로 JOIN하여, 각 주문에 대한 고객 이름과 등급을 함께 출력하세요.

In [ ]:
# Q6 정답

conn = sqlite3.connect('customers.db')

# INNER JOIN으로 두 테이블 결합
query = """
SELECT 
    o.주문ID,
    c.이름,
    c.등급,
    o.주문금액,
    o.주문일
FROM orders o
JOIN customers c ON o.고객ID = c.고객ID
ORDER BY o.주문일
"""

df_joined = pd.read_sql(query, conn)
conn.close()

print("주문 내역 (고객 정보 포함):")
print(df_joined)

In [ ]:
# 테스트
assert len(df_joined) == 6, "6건의 주문이 있습니다"
assert '이름' in df_joined.columns, "고객 이름이 포함되어야 합니다"
assert '등급' in df_joined.columns, "고객 등급이 포함되어야 합니다"
assert df_joined[df_joined['주문ID'] == 1001]['이름'].values[0] == '김철수', "주문ID 1001은 김철수의 주문입니다"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. `FROM orders o` - orders 테이블에 별칭 `o` 부여
2. `JOIN customers c` - customers 테이블에 별칭 `c` 부여
3. `ON o.고객ID = c.고객ID` - 조인 조건 (두 테이블의 공통 키)
4. `o.주문ID`, `c.이름` - 테이블 별칭으로 컬럼 구분

**핵심 개념**:
- INNER JOIN: 양쪽 테이블에 모두 존재하는 행만 결합 (pandas의 `merge(how='inner')`)
- 테이블 별칭(Alias): 쿼리를 간결하게 작성
- JOIN은 관계형 DB의 핵심 기능

**대안 솔루션**:
```python
# pandas merge로 동일한 결과
df_orders = pd.read_sql("SELECT * FROM orders", conn)
df_customers = pd.read_sql("SELECT * FROM customers", conn)
df_joined = df_orders.merge(df_customers, on='고객ID', how='inner')
```

**흔한 실수**:
- ❌ ON 절 누락: `FROM orders JOIN customers` → 카르테시안 곱 발생
- ❌ 컬럼 이름 중복: `SELECT 고객ID, ...` → 어느 테이블의 고객ID인지 불명확

**실무 팁**:
대용량 DB에서는 SQL JOIN이 pandas merge보다 빠르지만, 복잡한 변환은 pandas가 유리합니다.

---

### Q7. SQLAlchemy Engine 생성 ⭐⭐

**문제**: SQLAlchemy를 사용하여 `customers.db` 파일에 대한 Engine을 생성하고, `customers` 테이블의 데이터를 읽어오세요.

In [ ]:
# Q7 정답

from sqlalchemy import create_engine

# 1. SQLAlchemy Engine 생성
engine = create_engine('sqlite:///customers.db')

# 2. Engine으로 데이터 읽기
df = pd.read_sql_table('customers', engine)

print("SQLAlchemy로 읽은 고객 데이터:")
print(df)

In [ ]:
# 테스트
assert isinstance(df, pd.DataFrame), "결과는 DataFrame이어야 합니다"
assert df.shape == (5, 4), "5행 4열의 DataFrame이어야 합니다"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. `create_engine()`으로 Connection String 전달
2. SQLite는 `sqlite:///파일명.db` 형식 (슬래시 3개)
3. `pd.read_sql_table()`은 테이블명만 전달 (쿼리 불필요)

**핵심 개념**:
- SQLAlchemy는 다양한 DB를 통일된 방식으로 다루는 Python 표준 라이브러리
- Engine은 연결 풀(Connection Pool)을 관리하여 효율적

**대안 솔루션**:
```python
# pd.read_sql()로 쿼리 실행 (더 유연함)
df = pd.read_sql("SELECT * FROM customers", engine)
```

**흔한 실수**:
- ❌ 슬래시 개수 오류: `sqlite://customers.db` (슬래시 2개) → 에러
- ❌ 절대 경로 사용: `sqlite:////Users/.../customers.db` (슬래시 4개)

**실무 팁**:
PostgreSQL이나 MySQL로 전환할 때는 Connection String만 바꾸면 되므로, 코드 재사용성이 높습니다.

---

### Q8. Context Manager로 안전한 쿼리 실행 ⭐⭐⭐

**문제**: SQLAlchemy Engine과 `with` 문을 사용하여, 등급별 고객 수를 계산하는 쿼리를 실행하세요.

In [ ]:
# Q8 정답

from sqlalchemy import create_engine

engine = create_engine('sqlite:///customers.db')

# Context Manager로 자동 연결 관리
with engine.connect() as conn:
    query = """
    SELECT 
        등급,
        COUNT(*) as 고객수
    FROM customers
    GROUP BY 등급
    ORDER BY 등급
    """
    df_grade = pd.read_sql(query, conn)

# with 블록을 벗어나면 자동으로 연결 종료
print("등급별 고객 수:")
print(df_grade)

In [ ]:
# 테스트
assert len(df_grade) == 3, "3가지 등급이 있습니다"
assert df_grade[df_grade['등급'] == 'Gold']['고객수'].values[0] == 2, "Gold 등급은 2명입니다"
assert df_grade[df_grade['등급'] == 'Silver']['고객수'].values[0] == 2, "Silver 등급은 2명입니다"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. `with engine.connect() as conn:` - Context Manager로 연결 생성
2. with 블록 내에서 쿼리 실행
3. 블록을 벗어나면 자동으로 `conn.close()` 호출됨

**핵심 개념**:
- Context Manager (`with` 문)는 리소스 관리를 자동화
- 예외 발생 시에도 반드시 연결이 종료됨 (안전성)
- Python의 `__enter__`, `__exit__` 매직 메소드 활용

**대안 솔루션**:
```python
# try-finally로 수동 관리 (비권장)
conn = engine.connect()
try:
    df = pd.read_sql(query, conn)
finally:
    conn.close()
```

**흔한 실수**:
- ❌ `with` 없이 연결 사용 후 `close()` 누락 → 연결 누수
- ❌ `with` 블록 밖에서 `conn` 사용 → 이미 닫힌 연결 에러

**실무 팁**:
프로덕션 코드에서는 항상 Context Manager를 사용하여 안전하게 DB 연결을 관리하세요.

---

### Q9. 데이터 수집 → DB 저장 → 분석 워크플로우 ⭐⭐⭐⭐

**문제**: 웹 스크레이핑 시뮬레이션 → DB 저장 → 분석의 3단계 워크플로우를 구현하세요.

In [ ]:
# Q9 정답

# 단계 1: 웹 스크레이핑 시뮬레이션 (실제로는 requests + BeautifulSoup)
reviews = pd.DataFrame({
    '제품ID': [201, 201, 202, 202, 203, 203, 204, 204],
    '평점': [5, 4, 3, 5, 4, 4, 2, 5],
    '리뷰': ['훌륭함', '좋음', '보통', '최고', '만족', '추천', '별로', '강추']
})

print("단계 1: 데이터 수집 완료")
print(reviews)

# 단계 2: DB 저장
conn = sqlite3.connect('reviews.db')
reviews.to_sql('product_reviews', conn, if_exists='replace', index=False)
conn.close()

print("\n단계 2: DB 저장 완료")

# 단계 3: 분석 (제품별 평균 평점, 4점 이상 필터링)
conn = sqlite3.connect('reviews.db')

query = """
SELECT 
    제품ID,
    AVG(평점) as 평균평점,
    COUNT(*) as 리뷰수
FROM product_reviews
GROUP BY 제품ID
HAVING AVG(평점) >= 4.0
ORDER BY 평균평점 DESC
"""

df_result = pd.read_sql(query, conn)
conn.close()

print("\n단계 3: 분석 결과 (평점 4점 이상)")
print(df_result)

In [ ]:
# 테스트
assert len(df_result) == 3, "평점 4점 이상인 제품은 3개입니다"
assert df_result.iloc[0]['제품ID'] == 201, "제품ID 201의 평점이 가장 높습니다"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. **데이터 수집**: DataFrame 생성 (실제로는 API/크롤링)
2. **DB 저장**: `to_sql()`로 SQLite에 저장 (데이터 누적)
3. **분석**: SQL 집계 쿼리로 인사이트 도출

**핵심 개념**:
- **HAVING 절**: GROUP BY 후 집계 결과에 조건 적용 (WHERE는 행 단위, HAVING은 그룹 단위)
- Week 2-3 프로젝트의 핵심 워크플로우
- DB는 데이터 저장소 역할, pandas는 분석 도구 역할

**대안 솔루션**:
```python
# pandas로 분석 (SQL 대신)
df = pd.read_sql("SELECT * FROM product_reviews", conn)
df_result = df.groupby('제품ID')['평점'].mean().reset_index()
df_result = df_result[df_result['평점'] >= 4.0]
```

**흔한 실수**:
- ❌ WHERE로 평균 필터링: `WHERE AVG(평점) >= 4.0` → 집계 함수는 HAVING에서만 사용
- ❌ 데이터 누적 미고려: `if_exists='replace'`만 사용 → 이전 데이터 삭제

**실무 팁**:
Week 2-3 프로젝트에서는 매일 크롤링한 데이터를 `if_exists='append'`로 누적 저장하면 시계열 분석이 가능합니다.

---

### Q10. 복합 집계 및 시각화 ⭐⭐⭐⭐⭐

**문제**: `customers.db`의 `customers`와 `orders` 테이블을 활용하여 등급별 총 주문금액과 평균 주문금액을 계산하고 Plotly로 시각화하세요.

In [ ]:
# Q10 정답

import plotly.graph_objects as go
from sqlalchemy import create_engine

# 1. SQL 집계 쿼리
engine = create_engine('sqlite:///customers.db')

with engine.connect() as conn:
    query = """
    SELECT 
        c.등급,
        SUM(o.주문금액) as 총주문금액,
        AVG(o.주문금액) as 평균주문금액,
        COUNT(o.주문ID) as 주문건수
    FROM customers c
    JOIN orders o ON c.고객ID = o.고객ID
    GROUP BY c.등급
    ORDER BY 총주문금액 DESC
    """
    df_grade_sales = pd.read_sql(query, conn)

print("등급별 주문 분석:")
print(df_grade_sales)

In [ ]:
# 2. Plotly Grouped Bar Chart 시각화
fig = go.Figure()

# 총 주문금액 바
fig.add_trace(go.Bar(
    x=df_grade_sales['등급'],
    y=df_grade_sales['총주문금액'],
    name='총 주문금액',
    marker_color='royalblue',
    text=df_grade_sales['총주문금액'],
    texttemplate='%{text:,.0f}원',
    textposition='outside'
))

# 평균 주문금액 바
fig.add_trace(go.Bar(
    x=df_grade_sales['등급'],
    y=df_grade_sales['평균주문금액'],
    name='평균 주문금액',
    marker_color='lightcoral',
    text=df_grade_sales['평균주문금액'],
    texttemplate='%{text:,.0f}원',
    textposition='outside'
))

fig.update_layout(
    title='등급별 주문금액 분석 (총 vs 평균)',
    xaxis_title='고객 등급',
    yaxis_title='금액 (원)',
    barmode='group',
    height=500,
    showlegend=True,
    template='plotly_white'
)

fig.show()

### 3. 분석 결과

**인사이트**:
1. **Gold 등급**: 총 주문금액이 가장 높지만(350,000원), 주문 건수는 2건
2. **Silver 등급**: 평균 주문금액은 낮지만(87,500원), 총 주문금액은 두 번째(175,000원)
3. **Bronze 등급**: 주문 건수가 적어 총 주문금액이 가장 낮음(50,000원)

**결론**: 
- Gold 등급 고객이 **가장 높은 구매력**을 보임 (평균 175,000원/건)
- Silver 등급은 **거래 빈도는 높지만 건당 금액은 낮음**
- Bronze 등급 고객의 **활성화 전략**이 필요함

**실무 활용**:
- Gold 고객에게 **프리미엄 상품 추천**
- Silver 고객에게 **번들 할인** 제공 (평균 주문금액 증대)
- Bronze 고객에게 **첫 구매 할인** 프로모션

In [ ]:
# 테스트
assert len(df_grade_sales) == 3, "3가지 등급이 있습니다"
assert df_grade_sales.iloc[0]['등급'] == 'Gold', "Gold 등급의 총 주문금액이 가장 높습니다"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. **SQL JOIN + GROUP BY**: 두 테이블을 결합하여 등급별 집계
2. **Plotly Grouped Bar**: 총/평균을 나란히 비교
3. **데이터 기반 인사이트**: 숫자에서 비즈니스 의미 도출

**핵심 개념**:
- SQL은 집계, pandas/Plotly는 시각화 → 역할 분담
- Grouped Bar Chart는 카테고리별 여러 지표 비교에 적합
- 분석 결과를 비즈니스 액션으로 연결

**대안 솔루션**:
```python
# Plotly Express 사용 (더 간결)
import plotly.express as px
df_melted = df_grade_sales.melt(id_vars='등급', value_vars=['총주문금액', '평균주문금액'])
fig = px.bar(df_melted, x='등급', y='value', color='variable', barmode='group')
```

**흔한 실수**:
- ❌ JOIN 없이 별도 집계: 등급과 주문을 따로 계산 → 매칭 불가
- ❌ 시각화만 제공: 인사이트 및 액션 플랜 부재

**실무 팁**:
Week 2-3 프로젝트 발표에서 이런 복합 분석 + 시각화 + 인사이트 패턴을 활용하면 높은 평가를 받을 수 있습니다.

---

## 📊 학습 정리

### Part 1: 기초 핵심 요약

| 개념 | 핵심 메소드/문법 | 실무 활용 |
|------|----------------|----------|
| **SQLite 연결** | `sqlite3.connect('파일명.db')` | 파일 기반 DB로 프로토타입 제작 |
| **DataFrame → SQL** | `df.to_sql('테이블명', conn, if_exists='replace')` | 크롤링/API 데이터를 DB에 저장 |
| **SQL → DataFrame** | `pd.read_sql('SELECT ...', conn)` | SQL 쿼리 결과를 pandas로 분석 |
| **WHERE 필터링** | `WHERE 컬럼명 >= 값` | 대용량 데이터 필터링 (메모리 절약) |
| **GROUP BY 집계** | `GROUP BY 컬럼명` + `SUM()`, `COUNT()` | DB 엔진의 빠른 집계 활용 |

### Part 2: 심화 핵심 요약

| 기법 | 문법 | 언제 쓰나? |
|------|------|----------|
| **SQLAlchemy** | `create_engine('sqlite:///파일명.db')` | 다양한 DB에 통일된 방식으로 연결 |
| **Context Manager** | `with engine.connect() as conn:` | 자동 연결 종료로 안전한 DB 작업 |
| **pandas-gbq** | `pandas_gbq.read_gbq(query, project_id)` | BigQuery 대용량 데이터 샘플링 |
| **JOIN 쿼리** | `SELECT * FROM A JOIN B ON A.id = B.id` | 여러 테이블의 데이터 결합 |

### 💡 실무 팁

1. **SQL vs pandas 역할 분담**:
   - SQL: 대용량 데이터 필터링, 집계, JOIN (DB 엔진의 최적화 활용)
   - pandas: 세밀한 분석, 복잡한 변환, 시각화 (Python의 유연성 활용)

2. **SQLite 활용 시나리오**:
   - 프로토타입 개발 (별도 서버 불필요)
   - 소규모 프로젝트 (수백만 건 이하)
   - Git으로 데이터 공유 (파일 하나로 완결)

3. **BigQuery 활용 시나리오**:
   - 페타바이트급 로그 데이터 분석
   - 실시간 대시보드 (초고속 집계)
   - 팀 협업 (클라우드 기반)

4. **Connection String 관리**:
   - 환경변수로 관리: `os.getenv('DATABASE_URL')`
   - `.env` 파일 사용 (Git에 업로드 금지)
   - 비밀번호 노출 방지

5. **에러 처리**:
   - `try-except`로 DB 에러 핸들링
   - `finally`로 연결 종료 보장
   - `with` 문 사용 권장 (자동 종료)

---

**다음 학습**: Day 6-3 통합 실습 - 다중 소스 데이터 분석 (웹 스크레이핑 → DB 저장 → pandas 분석 → Plotly 시각화)